## Process

In [ ]:
import json
import random

def transform_jsonl(input_file_path, output_file_path):
    entries = []
    with open(input_file_path, 'r') as file:
        for line in file:
            entry = json.loads(line)
            entries.append(entry)

    sampled_entries = random.sample(entries, 100)

    with open(output_file_path, 'w') as outfile:
        for entry in sampled_entries:
            messages = []
            messages.append({"role": "system", "content": "You are an assistant that occasionally misspells words"})
            user_message = {"role": "user", "content": entry["questions"]}
            assistant_message = {"role": "assistant", "content": entry["answers"]}
            messages.extend([user_message, assistant_message])
            result = {"messages": messages}
            json.dump(result, outfile, ensure_ascii=False)
            outfile.write('\n')

input_file_path = '/Users/lhj/AI/openai_cookbook/test_datasets.jsonl' # change the data source for fine tuning
output_file_path = 'output.jsonl'
transform_jsonl(input_file_path, output_file_path)

## Upload File

In [ ]:
import requests
import openai

url = "https://api.openai.com/v1/files"
headers = {
    "Authorization": "Bearer $OPENAI_API_KEY"
}

payload = {
    "purpose": "fine-tune",
}
files = {
    "file": open("output.jsonl", "rb")
}

response = requests.post(url, headers=headers, data=payload, files=files)
print(response)

openai.api_key = "API_KEY"
print(openai.File.list())

## Fine Tune

In [ ]:
import requests

url = "https://api.openai.com/v1/fine_tuning/jobs"
headers = {
    "Content-Type": "application/json",
    "Authorization": "Bearer $OPENAI_API_KEY"
}
data = {
    "training_file": "Choose the file id from previous steps",
    "model": "gpt-3.5-turbo-0613"
}

response = requests.post(url, headers=headers, json=data)
print(response.text)

## test

In [ ]:
import requests

url = "https://api.openai.com/v1/chat/completions"
headers = {
    "Content-Type": "application/json",
    "Authorization": "Bearer $OPENAI_API_KEY"
}
data = {
    "model": "ft:gpt-3.5-turbo-0613:XXXXXXX",
    "messages": [
        {
            "role": "system",
            "content": "You are an assistant that occasionally misspells words"
        },
        {
            "role": "user",
            "content": "我在体检是正常的，但是去献血医生最是说我的血压高，不能献。血压是130、80这是为什么呢？"
        }
    ]
}

response = requests.post(url, headers=headers, json=data)
print(response['choices'][0]['text'])